# This is a "Jupyter Notebook". 
It is a way of combining text and code (here we use the programming language Python3) and running it in a web browser. Coding is an incredible tool for exploring large amounts of data, especially if you want to perform similar analyses over and over on different datasets. 

We would like you to use this notebook as a basic calculator to get used to this environment. You will find it extremely useful for future work. 

We've put in the first few steps for you. 

some useful links for help with jupyter notebooks:

https://towardsdatascience.com/jypyter-notebook-shortcuts-bf0101a98330

some useful links for help with python:

https://howchoo.com/g/m2vmotm5odn/basic-math-in-python

https://pythonprogramming.net/math-basics-python-3-beginner-tutorial/

https://en.wikibooks.org/wiki/Non-Programmer%27s_Tutorial_for_Python_3

#### maths usage
| operator | name   | example |
|:------|:---------------|:------|
|   +  | addition| 2 + 2 |
|   -  | subtraction| 2 - 2 |
|   *  | multiplication| 2 * 2 |
|   /  | division| 2 / 2 |
|   **  | exponent| 2**4 |


#### Jupyter usage
| operator | name  |
|:------|:---------------|
|   shift + enter  | run a cell |
|   up/down arrows  | move between cells |

# Start coding :)

First, you will need to run this section of imports

In [1]:
import math
import numpy as np
import scipy
import scipy.integrate as integ
import astropy.units as u
from ipywidgets import interact, interactive, fixed, interact_manual


### As before, please run the following functions as they are.
If you wish to investigate how they work further, just ask :)

In [14]:
KT_TNT_EQUIV = 4.185e12
Grav_const = 6.6726e-11 #N-m2/kg2

def grav(alt, M, r):
    return Grav_const * M / (r + alt) ** 2

def atm_roh(alt, p0, scale_height):
    if p0>0:
        return p0 * np.exp(-alt / scale_height)
    else: return 0.

def new_alt(alt, angle, dt, v):
    dh = np.sin(angle) * dt * v
    return alt - dh

def rad2mass(rad, pm):
    return 4. / 3. * math.pi * (rad)**3 * pm

def find_strength(density):
    if density < 2000.:
        return 1e5
    elif density < 3000:
        return 1e6
    elif density < 4000:
        return 1e7
    else: return 1e8

### ablation and drag function definitions

In [40]:
def ablation_fun(v0, slope, r0, pm, yield_str, params, ground=0.):
    
    M = params['mass_planet']
    R =  params['mean_radius_planet']
    P0 =  params['atm_density_groundlevel']
    H = params['scale_height_planet']

    ## assumptions:
    ### Drag coefficient
    cd = 1.3
    
    ### Ablation coefficient
    sigma = 2.5e-8 # sec2/m2
    
    ## Start modelling from alt0 (~space)
    alt0 = 200000.0
    
    ## bulk strength equation from collins et al. 2005
    #yield_str = pow(10, 2.107+0.0624*pow(pm, 0.5))

    ## initiate breakup altitudes
    z_break = 0.
    z_ab = 0.
    z_b = 0.

    ## define common drag/ablation equation parameters
    ### constant wrt height for velocity
    kv = 3/8. * cd / pm 
    
    ### constant wrt height for radius
    kr = 1./8 * sigma * cd / pm 

    # I = 4.07 * cd * H * yield_str / (pm * d * v0**2 * sin(angle))
    # z_break = - H * (log(yield_str / (po * v0**2)) + 1.308 - 0.314 * I - 1.303 * np.sqrt(1 - I))
    
    ## mass for a sphere from volume and density
    m = 4. / 3. * math.pi * (r0)**3 * pm
    
    ## energy given as kinetic energy
    energy_0 = 0.5 * m * v0**2
    
    ## integration height range to be considered
    alt_range = [alt0, z_break]
    calc_alts=np.r_[alt0:ground:-100]
    
    ## parameters to be passed to integration:
    ### initialisation of state before hitting the atmosphere 
    ### [initial velocity, initial radius, initial energy, incoming angle]
    s_angle = np.sin(slope * u.deg)
    X = np.array([v0, r0,  energy_0, s_angle])
    X_all = np.array([v0, r0,  energy_0, s_angle, alt0])

    ### constants and meteoroid density
    param = [kv, kr, pm, M, R, P0, H]
    
    ## start integration. Stop when hits ground, when smaller than 1 mm diam or reach breakup altitude
    i = 1

    disintegrate_rad = 0.005
    
    while i<len(calc_alts) and X[1]>disintegrate_rad and calc_alts[i] > z_break:    

        ode_out = scipy.integrate.odeint(integrals_abl, X, [calc_alts[i-1], calc_alts[i]],  args = (param,)) 
        
        X = ode_out[1]

        ## calculate head pressure body is exerted to 
        z_test = atm_roh(calc_alts[i], P0, H) * X[0]**2

        ## test if the bulk strength of body is exceeded by head pressure. 
        ## set z_break to current altitude 
        if z_test>=yield_str:
            z_break = (calc_alts[i-1]+calc_alts[i])/2.0
        elif X[1] < disintegrate_rad:
            z_ab = calc_alts[i]
        
        save_row = np.hstack((X, calc_alts[i]))
        X_all = np.vstack((X_all, save_row))
                    
        i+= 1
        
    ## final state pulled from integration
    [v, r, energy, s_angle] = X
    slope = np.arcsin(s_angle) * u.rad
    ## mass for a sphere from volume and density
    m = 4. / 3. * math.pi * (r)**3 * pm
    energy = 0.5 * m * v**2


    return [v,                            # velocity
           r,                            # radius
           energy,                       # final energy
           slope.to(u.degree),                        # final flight angle
           z_break,
            m,
            z_ab
           ]


In [41]:
## define the function to integrate wrt height
def integrals_abl(X, h, param): 
    [kv, kr, pm, M, R, P0, H] = param
    
    g = grav(h, M, R)
    [v, rad, energy, s_angle] = X

    m = rad2mass(rad, pm)

    Xdot=[0., 0., 0., 0.] 


    #dv/dh
    Xdot[0] = kv * atm_roh(h, P0, H) * v / (rad * s_angle) - g / v
    
    #dr/dh
    Xdot[1] = kr * atm_roh(h, P0, H) * v**2 / (s_angle)
    
    #dE/dh
    Xdot[2] = math.pi * pm * (2 * rad**2 * v**2 * Xdot[1] + 4/3. * rad**3 * v * Xdot[0]) #v**2 * m * v * Xdot[0]    
    
    #d_angle/dh
    Xdot[3] = -g / v**2 * (1 - s_angle**2) / s_angle

    return Xdot 

# Question 1

### Part 1: define different planet properties. 

Some have been filled in for you.

Use your notes from previous labs or google to fill in the missing values.

Take note of the units we're looking for. 

In [42]:
def planet_params(planet):
    
    if planet== 'Earth':
        mass_planet =                # kg 
        mean_radius_planet =         # metres
        atm_density_groundlevel =    # kg/m3
        scale_height_planet =        # m
        
    elif planet== 'Mars':
        mass_planet =                # kg 
        mean_radius_planet =         # metres
        atm_density_groundlevel =    # kg/m3
        scale_height_planet =        # m
        
    elif planet== 'Moon':
        mass_planet =                # kg 
        mean_radius_planet =         # metres
        atm_density_groundlevel =    # kg/m3
        scale_height_planet =        # m
        
    elif planet== 'Venus':
        mass_planet =                # kg 
        mean_radius_planet =         # metres
        atm_density_groundlevel =    # kg/m3
        scale_height_planet =        # m
        
        
    surface_gravity = Grav_const * mass_planet / mean_radius_planet**2
        
    return {'mass_planet':mass_planet, 
            'mean_radius_planet':mean_radius_planet, 
            'surface_gravity':surface_gravity,
            'atm_density_groundlevel':atm_density_groundlevel, 
            'scale_height_planet':scale_height_planet}


### Part 2

Run the cell below and follow the instructions it prints out.


In [46]:
print(f"\nFill in the following boxes. \nNote that we are using SI units, so meters, m/s, kg, kg/m3\n")

def impact(planet, initial_speed, angle, density, diameter):
    
    density = float(density)
    initial_speed = float(initial_speed)
    density = float(density)
    diameter = float(diameter)
    angle = float(angle)
    if angle < 0 or angle > 90:
        print('angle should be between 0 and 90. default of 45 being used.')
    
    params = planet_params(planet)
    surface_gravity = params['surface_gravity']

    # radius of the body
    r0 = diameter/2.    
    print(f"\ninitial speed = {initial_speed:.2f} m/s \ninitial diameter = {diameter:.2f}; initial radius = {r0:.2f} m \ninitial mass = {rad2mass(r0, density):.2e} kg \ninitial energy = {0.5 * rad2mass(r0, density) * initial_speed**2:.2e} J \nimpact angle = {angle:.2f} \nmeteoroid density = {density}")
    print(f"\nIf this were to hit {planet}")
    

    
    # how does the atmosphere affect the body?
    # first check strength based on meteoroid type from density
    bulk_strength = find_strength(density)
    rock_changes = ablation_fun(initial_speed, angle, r0, density, bulk_strength, params)

    final_speed = rock_changes[0]
    final_diameter = rock_changes[1] *  2.
    final_energy = rock_changes[2]
    final_slope = rock_changes[3]
    breakup_height = rock_changes[4]
    final_mass = rock_changes[5]    
    disintegrate_height = rock_changes[6]
    
    if breakup_height >0:
        print(f"it would not make a crater")
        print(f"and would break up at {breakup_height /1000:.2f} km")
        
    elif disintegrate_height > 0.:
        print(f"it would not make a crater")
        print(f"and would entirely burn up in the atmosphere at {disintegrate_height/1000:.2f} km")
    else:
        # determine the size of the crater:
        a = 8.8e-3
        b = 0.32
        
        D = final_energy**b * a * pow(9.8/surface_gravity, 3./16)
        crater_diameter = D

        print(f"it would drop a meteorite on the ground!")
        if final_diameter>crater_diameter:
            print(f"but it would not make a crater")
        else:
            print(f"it would make a crater {crater_diameter:.2f} m in diameter")
                
    
    print(f"\nfinal speed = {final_speed:.2f} m/s \nfinal diameter = {final_diameter:.4f} m \nfinal energy = {final_energy:.2e} J \nfinal slope = {final_slope:.2f} \nfinal mass = {final_mass:.4f} kg")
#     print(f"estimate: {crater_diameter:.1f} m crater on Mars \n {e_TNT:.1e} kt TNT impact energy \n {impactor_mass:.1e} kg impactor mass \n {impactor_diameter:.2f} m impactor diameter')
    
interact(impact, planet=['Earth', 'Mars', 'Moon', 'Venus'], initial_speed='1.', angle='45', density='1.         ', diameter='1.')




Fill in the following boxes. 
Note that we are using SI units, so meters, m/s, kg, kg/m3



interactive(children=(Dropdown(description='planet', options=('Earth', 'Mars', 'Moon', 'Venus'), value='Earth'…

<function __main__.impact(planet, initial_speed, angle, density, diameter)>